In [1]:
import numpy as np
import tensorflow as tf


In [2]:
xs = np.array([5.0, 6.0, 7.0, 8.0 ,9.0, 10.0], dtype=float)
ys = np.array([9.0, 11.0, 13.0, 15.0, 17.0, 19.0], dtype=float)

In [7]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape =[1])])
model.compile(optimizer= 'sgd', loss = 'mean_squared_error')

In [8]:
model.fit(xs, ys, epochs=550, verbose=0 )

In [9]:
print(model.predict([4.0]))

1/1 [==============================] - 0s 74ms/step
[[7.401493]]


In [14]:
# def myHuber(yt, yp):
#   threshold = 1
#   error = yt - yp
#   errorAbs = tf.abs(error)
#   if errorAbs<= threshold:
#     result =tf.square(error)/2
#   else:
#     result = threshold*(errorAbs - (threshold/2))
#   return tf.convert_to_tensor(result, tf.float32)

Ok the code in the upper cell is not working. getting a lot of shape errors and I did not find any reason. logicaly it should work but it's not working. so I just accept the code bellow.\

In [16]:
def myHuber(yt, yp):
  threshold = 1
  error = yt - yp
  errorAbs = tf.abs(error)
  isSmall = errorAbs < threshold
  small = tf.square(error)/2
  big = threshold *(errorAbs - (threshold/2))
  return tf.where(isSmall, small, big)

In [ ]:
xs = xs.reshape(6, 1)
ys = ys.reshape(6, 1)

In [ ]:
print(np.shape(xs))
print(tf.shape(xs))

(6,)
tf.Tensor([6], shape=(1,), dtype=int32)


In [17]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape= [1])])
model.compile(optimizer='sgd', loss = myHuber)
model.fit(xs, ys, epochs = 550, verbose = 0)
print(model.predict([[4.0]]))

1/1 [==============================] - 0s 54ms/step
[[7.526476]]


Time to code with class and More HyperParameters


In [18]:
def myHuberWithThreshold(threshold):
  def myHuber(trueVal, predVal):
    error = trueVal - predVal
    errorAbs = tf.abs(error)
    isSmall = errorAbs < threshold
    small = tf.square(error)/2
    big  = threshold * (errorAbs - (threshold/2))
    return tf.where(isSmall, small, big)
  return myHuber
  


In [21]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape= [1])])
model.compile(optimizer='sgd', loss = myHuberWithThreshold(threshold = 10))
model.fit(xs, ys, epochs = 550, verbose = 0)
print(model.predict([[4.0]]))

1/1 [==============================] - 0s 54ms/step
[[7.4517355]]


In [7]:
class myHuberClass(tf.keras.losses.Loss):
  threshold = 1
  def __init__(self, threshold):
    super().__init__()
    self.threshold = threshold
  def call (self, yt, yp):
    error = yt - yp
    errorAbs = tf.abs(error)
    isSmall = errorAbs < self.threshold
    small = tf.square(error)/2
    big  = self.threshold * (errorAbs - (self.threshold/2))
    return tf.where(isSmall, small, big)

In [11]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape= [1])])
model.compile(optimizer='sgd', loss = myHuberClass(threshold = 3.2))
model.fit(xs, ys, epochs = 550, verbose = 0)
print(model.predict([[4.0]]))

1/1 [==============================] - 0s 59ms/step
[[7.5236907]]


ok now we're almost done! 
the Contrastive Loss Function is the last thing that we should implement

In [24]:
class myCLFclass(tf.keras.losses.Loss):
  margin = 1
  def __init__(self, margin):
    super().__init__()
    self.margin = margin
  def call(self, yt, yp):
    yp2 = tf.square(yp)
    maxMargin2 = tf.square(tf.maximum(self.margin-yp, 0))
    result = yt + yp2 +(1-yt)*maxMargin2
    return tf.reduce.mean(result)

In [25]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape= [1])])
model.compile(optimizer='sgd', loss = myCLFclass(margin = 1))
model.fit(xs, ys, epochs = 550, verbose = 0)
print(model.predict([[4.0]]))

AttributeError: ignored

and again we have a problem here.
I don't know how to get the overal mean of result variable in the code and gpt is not answering correctly
the problem is that when I don't get the overal mean it is returnin a NaN value
actually it prints the NaN value
